In [1]:
import os
import cfg
import csv
import pandas as pd

import pprint

In [2]:
meta = pd.read_csv("meta_climate.csv")

# remove duplicates
meta = meta.drop_duplicates(subset=["filename"]).reset_index(drop=True)

meta.head(3)

,filename,uuid,subquery,time,text,permalink
0,2022-12-31_2015_US_MSNBC_Morning_Joe.txt,ce432176-8947-11ed-94e9-089e01ba034d,environment,2022-12-31 20:47:17,IS REALLY HARD FOR\nCOMPANIES ACROSS THE COUNT...,"http://www.sscnet.ucla.edu/tna/edge/video,ce43..."
1,2022-12-31_2000_US_FOX-News_The_Journal_Editor...,b5f7ce7a-8945-11ed-b9fc-089e01ba0338,fossil,2022-12-31 20:14:24,BEG VENEZUELA WE\nDO NOT INCREASE PRODUCTION H...,"http://www.sscnet.ucla.edu/tna/edge/video,b5f7..."
2,2022-12-31_1700_US_CNN_CNN_Newsroom_With_Fredr...,91c395a2-892c-11ed-b597-089e01ba0338,environment,2022-12-31 17:20:55,RIGHT AMERICAN POLITICS.\nWHEN IT COMES TO ABO...,"http://www.sscnet.ucla.edu/tna/edge/video,91c3..."


In [3]:
meta.shape

(88232, 6)

In [4]:
cues, data = ["|CC1|", "|SEG_00|", "Type=Commercial", "Type=Story start"], []

for ind in range(10):
    try:
        ent = meta.iloc[ind].filename.split("-")

        # date entities
        path = [ent[0], ent[1], "-".join(ent[2 : ]).split("_")[0], "_".join("-".join(ent[2 : ]).split("_")[1 : ])]
        # filepath
        f_path = os.path.join(cfg.TV_PATH, path[0], "-".join(path[ : 2]), "-".join(path[ : 3]), meta.iloc[ind].filename)
        # read text file
        with open(f_path) as f: lines = f.read()

        stories, meta_stories = [story.split("\n")[1 : ] for story in lines.split(cues[1]) if cues[3] in story], []

        for story in stories:
            trs = " ".join([line.split(cues[0])[-1] for line in story]).replace(">>", "")

            if any([key.upper() in trs for key in cfg.KEYS["climate_change"]]):

                meta_story = {
                    "file_path": f_path, 
                    "trs": trs,
                    "start": story[0].split(cues[0])[0].split("|")[0],
                    "end": story[-1].split("|")[0]
                    }
                meta_stories.append(meta_story)
        
        data.extend(meta_stories)
    
    except: pass

In [8]:
import json

with open("climate.json", 'w') as f:
    json.dump(data, f, indent=4)